## Выбираем лучшие настройки для тикеров

In [2]:
%reload_ext autoreload
%autoreload 2

In [4]:
from app.cache import LocalCache
from bot.vis import TaskProgress
from bot import TestAlgorithm
from app.config import RunConfig
from concurrent.futures import ThreadPoolExecutor, as_completed

test_configs = [
    (RunConfig(
        ticker=t_config.ticker,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=1,

        majority_trade=t_config.majority_trade,
        pretest_period=pretest_period,
        pretest_type=t_config.pretest_type,

        threshold_buy_steps=0,
        threshold_sell_steps=0,
        stop_up_p=stop_up_p,
        stop_down_p=0,

        step_size=t_config.step_size + step_size_shift,
        step_set_orders_cnt=step_cnt,
    ))
    for t_config in [
        RunConfig.from_repr_string('MVID+ 6/pre3:-6/2 x l1 x 1.6¤ |u0.01 d0.0|'),
    ]
    # for max_shares in [3]
    # for base_shares in [0]
    # for stop_up_p in [0]
    # for step_size_shift in [0] # [0, .2, -.2, -.4, .4]
    # for step_cnt in [2]  # вот тут некоторым удобнее в 2. можно потестить в вариаторе 2 и 3
    # for pretest_period in [5]  # range(3, 7) # todo

    for max_shares in [3, 4]
    for base_shares in [0]
    for stop_up_p in [0, 0.01]
    for step_size_shift in [0, .2, -.2]
    # for step_size_shift in [0, .2, -.2, -.4, -.6, -.8]
    for step_cnt in [2]
    for pretest_period in range(3, 7)
]

# test_configs = [
#     RunConfig.from_repr_string('SPBE- 3/pre5:1/2 x l1 x 1.0¤'),
#     RunConfig.from_repr_string('SPBE- 3/pre5:0/2 x l1 x 1.0¤'),
# ]

def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config, use_cache=True)
    return test_alg.test(
        last_test_date=None,
        test_days_num=20,  # todo
        shares_count=0,

        auto_conf_days_freq=1,
        auto_conf_prev_days=config.pretest_period,
    )

unique_configs = set(test_configs)
# unique_configs = test_configs

results = []
LocalCache.clear()
progress = TaskProgress(len(unique_configs))


# todo на первом проходе надо 1 ставить, тогда запросы к API будут только 1 раз
with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in unique_configs}
    
    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()


print(f"cache {LocalCache.get_counter('cache_miss')} vals, used {LocalCache.get_counter('cache_find')} times")

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: (x['config'].ticker, -float(x['profit_p'])), reverse=False)

print()
for item in sorted_results:
    print(item)

Запуск в 00:11
Закончено в 00:15, количество 48, длительность 0:04:28                                              
cache 19259 vals, used 129973 times

{'profit': 92.0, 'profit_p': 13.85, 'profit_p_avg': 0.69, 'config': MVID+ 3/pre3:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'last_conf': MVID+ 6/pre3:-6/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 133}
{'profit': 90.0, 'profit_p': 13.55, 'profit_p_avg': 0.68, 'config': MVID+ 3/pre3:0/2 x l1 x 1.4000000000000001¤ |u0.01 d0.0| , 'last_conf': MVID+ 3/pre3:-3/2 x l1 x 0.8¤ |u0.01 d0.0| , 'op': 135}
{'profit': 87.7, 'profit_p': 13.2, 'profit_p_avg': 0.66, 'config': MVID+ 3/pre3:0/2 x l1 x 1.8¤ |u0.01 d0.0| , 'last_conf': MVID+ 6/pre3:-6/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 132}
{'profit': 103.9, 'profit_p': 11.73, 'profit_p_avg': 0.59, 'config': MVID+ 4/pre3:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'last_conf': MVID+ 6/pre3:-6/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 150}
{'profit': 97.2, 'profit_p': 10.98, 'profit_p_avg': 0.55, 'config': MVID+ 4/pre3:0/2 x l1 x 1.40000000000000